# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Dividir em treino (100 amostras) e teste (50 amostras)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    train_size=100,
    test_size=50,
    random_state=42  # para reprodutibilidade
)

## Definição do modelo e treinamento

In [ ]:
# 3. Converter para tensores PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# 4. Definir modelo: regressão logística
modelo = torch.nn.Linear(4, 1)  # 4 features → 1 saída

## Execução do treinamento

In [ ]:
# 5. Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(modelo.parameters(), lr=0.1)

# 6. Treino
print("=== Iniciando Treinamento ===\n")
for epoch in range(1000):
    # Modo de treino
    modelo.train()
    optimizer.zero_grad()  # reseta gradiente senão acumula
    outputs = modelo(X_train)
    loss = funcao_perda(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Avaliação no conjunto de teste a cada 100 épocas
    if (epoch + 1) % 100 == 0:
        modelo.eval()  # modo de avaliação
        with torch.no_grad():
            test_outputs = modelo(X_test)
            test_loss = funcao_perda(test_outputs, y_test)

            # Calcular probabilidades
            probabilities = torch.sigmoid(test_outputs)

            # Calcular acurácia no teste
            predictions = probabilities > 0.5
            accuracy = (predictions == y_test).float().mean()

            # Calcular AUC-ROC
            y_test_np = y_test.numpy()
            probabilities_np = probabilities.numpy()
            auc_roc = roc_auc_score(y_test_np, probabilities_np)

        print(f"Época [{epoch+1}/1000], Loss Treino: {loss.item():.4f}, "
              f"Loss Teste: {test_loss.item():.4f}, Acurácia: {accuracy.item():.4f}, "
              f"AUC-ROC: {auc_roc:.4f}")

# 7. Avaliação final e busca do melhor threshold
print("\n=== Avaliação Final ===\n")
modelo.eval()
with torch.no_grad():
    final_test_outputs = modelo(X_test)
    final_probabilities = torch.sigmoid(final_test_outputs)

    # Converter para numpy
    y_test_np = y_test.numpy().flatten()
    final_probabilities_np = final_probabilities.numpy().flatten()

    # Calcular curva ROC
    fpr, tpr, thresholds = roc_curve(y_test_np, final_probabilities_np)
    final_auc_roc = roc_auc_score(y_test_np, final_probabilities_np)

# 8. Encontrar o melhor threshold através da busca exaustiva
print("=== Buscando Melhor Threshold ===\n")

melhor_acuracia = 0
melhor_threshold = 0.5
melhor_vp = 0
melhor_vn = 0
melhor_fp = 0
melhor_fn = 0
resultados = []

# Testar diversos thresholds de 0.0 a 1.0
thresholds_teste = np.arange(0.0, 1.01, 0.01)

for threshold in thresholds_teste:
    # Fazer predições com o threshold atual
    predictions = (final_probabilities_np >= threshold).astype(int)

    # Calcular matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_test_np, predictions).ravel()

    # Calcular acurácia
    acuracia = (tp + tn) / (tp + tn + fp + fn)

    # Armazenar resultados
    resultados.append({
        'threshold': threshold,
        'acuracia': acuracia,
        'vp': tp,
        'vn': tn,
        'fp': fp,
        'fn': fn
    })

    # Atualizar melhor resultado
    if acuracia > melhor_acuracia:
        melhor_acuracia = acuracia
        melhor_threshold = threshold
        melhor_vp = tp
        melhor_vn = tn
        melhor_fp = fp
        melhor_fn = fn

# 9. Exibir resultados
print(f"Melhor Threshold: {melhor_threshold:.2f}")
print(f"Melhor Acurácia: {melhor_acuracia:.4f}")
print(f"\nMatriz de Confusão no Melhor Threshold:")
print(f"  Verdadeiros Positivos (VP): {melhor_vp}")
print(f"  Verdadeiros Negativos (VN): {melhor_vn}")
print(f"  Falsos Positivos (FP): {melhor_fp}")
print(f"  Falsos Negativos (FN): {melhor_fn}")
print(f"\nAUC-ROC: {final_auc_roc:.4f}")
print(f"Total de amostras de treino: {len(X_train)}")
print(f"Total de amostras de teste: {len(X_test)}")

# Calcular métricas adicionais
sensibilidade = melhor_vp / (melhor_vp + melhor_fn) if (melhor_vp + melhor_fn) > 0 else 0
especificidade = melhor_vn / (melhor_vn + melhor_fp) if (melhor_vn + melhor_fp) > 0 else 0
precisao = melhor_vp / (melhor_vp + melhor_fp) if (melhor_vp + melhor_fp) > 0 else 0

print(f"\nMétricas Adicionais:")
print(f"  Sensibilidade (Recall): {sensibilidade:.4f}")
print(f"  Especificidade: {especificidade:.4f}")
print(f"  Precisão: {precisao:.4f}")

# 10. Visualizações
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Gráfico 1: Curva ROC
axes[0].plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {final_auc_roc:.4f})')
axes[0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Classificador Aleatório')
axes[0].set_xlim([0.0, 1.0])
axes[0].set_ylim([0.0, 1.05])
axes[0].set_xlabel('Taxa de Falsos Positivos (FPR)')
axes[0].set_ylabel('Taxa de Verdadeiros Positivos (TPR)')
axes[0].set_title('Curva ROC')
axes[0].legend(loc="lower right")
axes[0].grid(alpha=0.3)

# Gráfico 2: Acurácia vs Threshold
acuracias = [r['acuracia'] for r in resultados]
thresholds_list = [r['threshold'] for r in resultados]
axes[1].plot(thresholds_list, acuracias, color='green', lw=2)
axes[1].axvline(x=melhor_threshold, color='red', linestyle='--',
                label=f'Melhor Threshold = {melhor_threshold:.2f}')
axes[1].axhline(y=melhor_acuracia, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Acurácia')
axes[1].set_title('Acurácia vs Threshold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Gráfico 3: Matriz de Confusão no Melhor Threshold
cm = np.array([[melhor_vn, melhor_fp], [melhor_fn, melhor_vp]])
im = axes[2].imshow(cm, cmap='Blues')

# Adicionar valores nas células
for i in range(2):
    for j in range(2):
        text = axes[2].text(j, i, cm[i, j], ha="center", va="center",
                           color="white" if cm[i, j] > cm.max()/2 else "black",
                           fontsize=20, fontweight='bold')

axes[2].set_xticks([0, 1])
axes[2].set_yticks([0, 1])
axes[2].set_xticklabels(['Predito: Negativo', 'Predito: Positivo'])
axes[2].set_yticklabels(['Real: Negativo', 'Real: Positivo'])
axes[2].set_title(f'Matriz de Confusão\n(Threshold = {melhor_threshold:.2f})')
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.show()

# 11. Mostrar top 5 melhores thresholds
print("\n=== Top 5 Melhores Thresholds ===")
resultados_ordenados = sorted(resultados, key=lambda x: x['acuracia'], reverse=True)
for i, r in enumerate(resultados_ordenados[:5], 1):
    print(f"{i}. Threshold: {r['threshold']:.2f} | Acurácia: {r['acuracia']:.4f} | "
          f"VP: {r['vp']}, VN: {r['vn']}, FP: {r['fp']}, FN: {r['fn']}")